This notebook goes through my workflow to extract features from the Groove MIDI Dataset

In [21]:
import numpy as np
import pretty_midi as pm
import pickle
from ganutilities import build_tone_maps, get_time_resolution

In [14]:
# Loads the metadata for the groove dataset
info = np.loadtxt("../groove/info.csv", delimiter=",", dtype="str")[1:]

In [15]:
# Builds tone map to convert discrete tones to indexes
tm = build_tone_maps()
tone_map = tm.get_tone_to_index()

In [16]:
# Split length and time resolution give ti
time_resolution = get_time_resolution()
split_length = 3200

In [17]:
# Splits a feature matrix into equal tick lengths
def split_sequence(seq):
    np.random.shuffle(seq)
    possible_batches = seq.shape[0] // split_length
    batches = np.split(seq[:(split_length*possible_batches), :, :], possible_batches, axis=0)
    return batches


In [18]:
# trims rows of leading -1's (silence) from beginning and end of feature matrix
def trim_zeros(seq):
    to_delete = []
    for i, row in enumerate(seq):
        if 1 not in row[:,0]:
            to_delete.append(i)
        else:
            break
    
    for i in range(seq.shape[0]):
        if 1 not in seq[-1 * i - 1][:,0]:
            to_delete.append(seq.shape[0] - i - 1)
        else:
            break
    
    np.delete(seq, to_delete, axis=0)

In [19]:
# converts midi file into feature matrix 
def convert_to_sequence(midi_data):
    global time_resolution
    
    duration = midi_data.get_end_time()
    total_ticks = int(duration / time_resolution) + 1
    
    note_sequence = np.full((total_ticks, 22, 2), -1, dtype='float')
    
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            
            tick_start = int(note.start / time_resolution)
            tick_end = int(note.end / time_resolution)
            
            # scales velocity from (-1 to 1)
            scaled_velocity = (note.velocity - 64) / 64
            
            # two channels for matrix, whether note exists and its velocity
            note_sequence[tick_start:tick_end, tone_map[note.pitch], 1] = scaled_velocity
            note_sequence[tick_start:tick_end, tone_map[note.pitch], 0] = 1
            
    return note_sequence

In [20]:
# loops through the metadata, and extracts features from midi files designated as fill (shorter sequences)
data = []

for i, meta in enumerate(info):
    split = meta[10]
    midi_path = meta[7]
    midi_data = pm.PrettyMIDI("../groove/" + midi_path)
    type = meta[5]
    if (split == "train" and type == "fill"):
        matrix = convert_to_sequence(midi_data)
        trim_zeros(matrix)
        if (matrix.shape[0] >= split_length):
            for batch in split_sequence(matrix):
                data.append(batch)
    
    print(i)
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [ ]:
data_array = np.stack(data)
print(data_array.shape)

(373, 3200, 22, 2)


In [ ]:
# saves feature matrices in a pickle file for easy reconstruction in training
with open('fillsplit3200.pickle', 'wb') as f:
    pickle.dump(data_array, f)

In [ ]:
with open('fillsplit1000.pickle', 'rb') as f:
    load = pickle.load(f)